In [2]:
import numpy as np 
import scipy.stats as sps
import pandas as pd
import h5py
import os
from tqdm import tqdm

from copy import deepcopy

from numpy import array
from os import environ
from os.path import join
from sys import argv

from glob import glob
from numpy import zeros
from os.path import basename, join

from keras.models import load_model
from os import environ
from os.path import abspath, dirname, join

from copy import copy

from sklearn.cross_validation import train_test_split

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.
/home/sergeymiller1996/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import keras
from keras import backend as K

from keras.models import Sequential
from keras.models import model_from_json

from keras.layers import InputLayer, Input, Dense, Activation, Dropout, BatchNormalization, Reshape
from keras.layers import Average, Conv2D, MaxPooling2D, Flatten
from keras.layers.merge import Concatenate
from keras.activations import relu, softmax, sigmoid
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model

from keras.regularizers import l2

from keras.constraints import maxnorm

from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical

from keras.losses import binary_crossentropy, mse

from keras.callbacks import LearningRateScheduler

import keras

In [4]:
import datagen_Miller
import models.keras_lstm
import models.dnn
import feature_generator

In [5]:
from importlib import reload

In [52]:
def score(y_true, y_pred):
    return 1 / (keras.losses.mse(y_true, y_pred)) ** 0.5

def run_model(X_train, y_train, X_val, y_val, 
          bad_columns=None,
          batch_size=1000,
          feature_expander=None,
          forecast_win=10,
          features_win=100,
          epochs=50,
          input_shape=None,
          path_to_load=None, 
          path_to_save=None,
          isLstm=False,
          epoch_train=100,
          epoch_val=100,
         ):
    
    assert isinstance(X_train, pd.DataFrame)
    assert input_shape is not None
    
    if path_to_load is None:
        if isLstm:
            model = models.keras_lstm.reccurent_model(
                                              input_shape=input_shape)
        else:
            model = models.dnn.nn_1(input_shape=input_shape)
    else:
        model = load_model(path_to_load)
        optimizer = Adam(1e-3, clipvalue=0.5)
        model.compile(loss='mean_squared_error', 
                      optimizer=optimizer, metrics=[score])
    
    datagen = datagen_Miller.DataGenerator(
                                        batch_size,
                                        feature_expander,
                                        forecast_win, 
                                        features_win,
                                        isLstm,
                                          )
    datagen.fit()
    
    filepath = "weights.{epoch:03d}-{val_loss:.3f}.hdf5"
    
    print('val steps {}'.format(X_val.shape[0] // batch_size))
    
    model.fit_generator(datagen.flow(X_train, y_train),
                        steps_per_epoch=min(epoch_train, X_train.shape[0] / batch_size), 
                        epochs=epochs, 
                        shuffle=False,
        callbacks=[
            datagen_Miller.TelegramCallback(),
            keras.callbacks.ModelCheckpoint(filepath, 
                                        monitor='val_loss', 
                                        verbose=0, 
                                        save_best_only=False, 
                                        save_weights_only=True, 
                                        mode='auto', 
                                        period=1)
                ],
        validation_data=datagen.flow(X_val, y_val),
        validation_steps=min(epoch_val, X_val.shape[0] // batch_size),
                       )
    
    if path_to_save:
        model.save_weights(path_to_save)

In [51]:
reload(datagen_Miller)
reload(models.keras_lstm)
reload(models.dnn)
reload(feature_generator)

<module 'feature_generator' from '/home/sergeymiller1996/docs/alpha/NightPy/feature_generator.py'>

In [9]:
%%time
# X_train_tea = pd.read_csv('splits/tea_full_train.csv')
# X_val_tea = pd.read_csv('splits/tea_full_validation.csv')
# X_train_coffee = pd.read_csv('splits/coffee_full_train.csv')
# X_val_coffee = pd.read_csv('splits/coffee_full_validation.csv')
X_test_tea = pd.read_csv('splits/tea_full_test.csv')
X_test_coffee = pd.read_csv('splits/coffee_full_test.csv')

CPU times: user 24 s, sys: 2.15 s, total: 26.2 s
Wall time: 27.3 s


In [10]:
filter_columns = ['Unnamed: 0', '0_ID', 'Y', '1_TIME']

In [11]:
y_test = np.array(X_test_tea['Y'])
X_test_tea = pd.DataFrame(X_test_tea.drop(filter_columns, axis=1), dtype=np.float32)
X_test_coffee = pd.DataFrame(X_test_coffee.drop(filter_columns, axis=1), dtype=np.float32)
# y_val = np.array(X_val_tea['Y'])
# X_val_tea = pd.DataFrame(X_val_tea.drop(filter_columns, axis=1), dtype=np.float32)
# X_val_coffee = pd.DataFrame(X_val_coffee.drop(filter_columns, axis=1), dtype=np.float32)

In [15]:
X_test_tea.shape[0]

3134616

In [13]:
final = pd.concat((X_test_tea, X_test_coffee), axis=1)

In [16]:
sz = int(0.95 * final.shape[0])

In [18]:
final_train, final_test = final.iloc[:sz], final.iloc[sz:]
y_train, y_test = y_test[:sz], y_test[sz:]

In [ ]:
%%time
try:
    run_model(final_train, y_train, 
              final_test, y_test, batch_size=200, epoch_train=100, epoch_val=400,
          feature_expander=None,
          input_shape=80,
          features_win=2,forecast_win=1, epochs=30, path_to_save='init_model', isLstm=True)
except KeyboardInterrupt:
    pass

val steps 783
Epoch 1/30
100/100 [==============================] - 79s 786ms/step - loss: 7.1952 - dense_23_loss: 0.0393 - lstm_13_loss_1: 0.0000e+00 - lstm_13_loss_2: 0.0000e+00 - dense_23_score: 469.6187 - dense_23_fake_loss_1: 0.0000e+00 - dense_23_fake_loss_2: 0.0000e+00 - lstm_13_score_1: 38.6640 - lstm_13_fake_loss_1: 0.0000e+00 - lstm_13_fake_loss_2: 0.0000e+00 - lstm_13_score_2: 12.1165 - lstm_13_fake_loss_3: 0.0000e+00 - lstm_13_fake_loss_4: 0.0000e+00 - val_loss: 2.1093 - val_dense_23_loss: 0.0390 - val_lstm_13_loss_1: 0.0000e+00 - val_lstm_13_loss_2: 0.0000e+00 - val_dense_23_score: 207.9584 - val_dense_23_fake_loss_1: 0.0000e+00 - val_dense_23_fake_loss_2: 0.0000e+00 - val_lstm_13_score_1: 39.8369 - val_lstm_13_fake_loss_1: 0.0000e+00 - val_lstm_13_fake_loss_2: 0.0000e+00 - val_lstm_13_score_2: 10.6272 - val_lstm_13_fake_loss_3: 0.0000e+00 - val_lstm_13_fake_loss_4: 0.0000e+00
Epoch 2/30
100/100 [==============================] - 74s 740ms/step - loss: 0.9215 - dense_23_lo

100/100 [==============================] - 74s 739ms/step - loss: 0.0314 - dense_23_loss: 0.0269 - lstm_13_loss_1: 0.0000e+00 - lstm_13_loss_2: 0.0000e+00 - dense_23_score: 522.3518 - dense_23_fake_loss_1: 0.0000e+00 - dense_23_fake_loss_2: 0.0000e+00 - lstm_13_score_1: 16.5568 - lstm_13_fake_loss_1: 0.0000e+00 - lstm_13_fake_loss_2: 0.0000e+00 - lstm_13_score_2: 0.1397 - lstm_13_fake_loss_3: 0.0000e+00 - lstm_13_fake_loss_4: 0.0000e+00 - val_loss: 0.0353 - val_dense_23_loss: 0.0303 - val_lstm_13_loss_1: 0.0000e+00 - val_lstm_13_loss_2: 0.0000e+00 - val_dense_23_score: 415.6011 - val_dense_23_fake_loss_1: 0.0000e+00 - val_dense_23_fake_loss_2: 0.0000e+00 - val_lstm_13_score_1: 16.2698 - val_lstm_13_fake_loss_1: 0.0000e+00 - val_lstm_13_fake_loss_2: 0.0000e+00 - val_lstm_13_score_2: 0.1380 - val_lstm_13_fake_loss_3: 0.0000e+00 - val_lstm_13_fake_loss_4: 0.0000e+00
Epoch 11/30
100/100 [==============================] - 74s 738ms/step - loss: 0.0359 - dense_23_loss: 0.0316 - lstm_13_loss_

100/100 [==============================] - 75s 755ms/step - loss: 0.0352 - dense_23_loss: 0.0315 - lstm_13_loss_1: 0.0000e+00 - lstm_13_loss_2: 0.0000e+00 - dense_23_score: 17198.0820 - dense_23_fake_loss_1: 0.0000e+00 - dense_23_fake_loss_2: 0.0000e+00 - lstm_13_score_1: 15.4119 - lstm_13_fake_loss_1: 0.0000e+00 - lstm_13_fake_loss_2: 0.0000e+00 - lstm_13_score_2: 0.1127 - lstm_13_fake_loss_3: 0.0000e+00 - lstm_13_fake_loss_4: 0.0000e+00 - val_loss: 0.0303 - val_dense_23_loss: 0.0270 - val_lstm_13_loss_1: 0.0000e+00 - val_lstm_13_loss_2: 0.0000e+00 - val_dense_23_score: 975.8430 - val_dense_23_fake_loss_1: 0.0000e+00 - val_dense_23_fake_loss_2: 0.0000e+00 - val_lstm_13_score_1: 15.5464 - val_lstm_13_fake_loss_1: 0.0000e+00 - val_lstm_13_fake_loss_2: 0.0000e+00 - val_lstm_13_score_2: 0.1257 - val_lstm_13_fake_loss_3: 0.0000e+00 - val_lstm_13_fake_loss_4: 0.0000e+00
Epoch 20/30
100/100 [==============================] - 74s 743ms/step - loss: 0.0344 - dense_23_loss: 0.0314 - lstm_13_los

 99/100 [============================>.] - ETA: 0s - loss: 0.0284 - dense_23_loss: 0.0256 - lstm_13_loss_1: 0.0000e+00 - lstm_13_loss_2: 0.0000e+00 - dense_23_score: 618.0022 - dense_23_fake_loss_1: 0.0000e+00 - dense_23_fake_loss_2: 0.0000e+00 - lstm_13_score_1: 15.0617 - lstm_13_fake_loss_1: 0.0000e+00 - lstm_13_fake_loss_2: 0.0000e+00 - lstm_13_score_2: 0.1002 - lstm_13_fake_loss_3: 0.0000e+00 - lstm_13_fake_loss_4: 0.0000e+00

In [ ]:
model = load_model(path_to_load)
        optimizer = Adam(1e-3, clipvalue=0.5)
        model.compile(loss='mean_squared_error', 
                      optimizer=optimizer)

In [ ]:
final = pd.concat((X_test_tea, X_test_coffee), axis=1)

In [ ]:
%%time
try:
    run_model(, y_train, 
              pd.concat((X_val_tea, X_val_coffee), axis=1), y_val, batch_size=1000, epoch_len=10,
          feature_expander=feature_generator.count_all_features,
          input_shape=262,
          features_win=2,forecast_win=1, epochs=10, path_to_save='init_model', isLstm=True)
except KeyboardInterrupt:
    pass

In [33]:
X_train.head()

,Unnamed: 0,0_ID,1_TIME,BID_P_1,BID_V_1,BID_P_2,BID_V_2,BID_P_3,BID_V_3,BID_P_4,...,ASK_P_6,ASK_V_6,ASK_P_7,ASK_V_7,ASK_P_8,ASK_V_8,ASK_P_9,ASK_V_9,ASK_P_10,ASK_V_10
0,5617875,TEA,1970.01.02 17:59:17.691677,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
1,5617876,TEA,1970.01.02 17:59:17.692349,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
2,5617877,TEA,1970.01.02 17:59:17.692369,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
3,5617878,TEA,1970.01.02 17:59:17.692587,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,105.0,2570.0,77.0,2571.0,124.0
4,5617879,TEA,1970.01.02 17:59:17.692607,2561,81,2560,73,2559,99,2558,...,2567,109.0,2568.0,92.0,2569.0,104.0,2570.0,77.0,2571.0,124.0


'2.0.9'